# This Notebook gives an example for using the implemented quantum circuit
In this example we will train the network on the MNIST data set for binary classification.
The task is to provide a classifier for numbers *0* and *4*.
We will train the network using the default parameters and evaulate the result.

## Usage
In this section the usage of the module is described.

### Imports
In the beginning we start with importing data_utils to create our train and test sets as well as the network itself.

In [ ]:
from quantum.utils import data_utils
from quantum.network import QuantumNetwork

### Data Set 
Now we can create the data set. We have to decide which dimensionality we will tackle. Increasing the size increases the amount ouf time we have to spent.
Using *Dimension=4* we have 16 pixels to consider. Since we want a binary calssifier we have to filter the values in the dataset, contradicting exampkles will also be removed.
We also have to define a mapping of Labesl to quantum states like `{label: 'state'}`.

In [ ]:
DIMENSION = 4
LABELS = {0: '1', 4: '0'}
(X_TRAIN, Y_TRAIN), (X_TEST, Y_TEST) = data_utils.generate_dataset(DIMENSION, filter_values=True, value_true=0, value_false=4)

### Network
In the next step we initialize the network. Important parameters are the dimensionality which influences the circuit and the labels.
Hyperparameters need to be set using the respective function. Default values are provided by the paper. If we want to use the 
efficient circuit (recommended) set `efficient=True` in the initialization. 

In [ ]:
NETWORK = QuantumNetwork(DIMENSION, LABELS, shots=512, efficient=True)
NETWORK.set_spsa_hyperparameters() # use this function to controll the SPSA algorithm

### Training
FInally we can start training. This is straight forward but time-consuming.
For now we set a small number of epochs.

In [ ]:
NETWORK.train_epochs(X_TRAIN, Y_TRAIN, epochs=2)

In [ ]:
NETWORK.print_stats()
test_count = 0
for sample, label in zip(X_TEST, Y_TEST):
    if (NETWORK.predict(sample) == label):
        test_count += 1
print(f'Test Accuracy: {test_count/X_TEST.shape[0]}')

### Loading and Saving
The model parameters can be stored and saved using the pickle format in python.

In [ ]:
filename = 'my_model.pickle'
NETWORK.save_model(filename)
NETWORK = QuantumNetwork.load_model(filename)

## Results
This section discusses the results of my findings.

### Perfomance
Qiskit is optimized for use of tensornetworks. This shows in the timing of the default circuit. 
The following results were archived on an AMD Ryzen 3700x (8x3.66 GHz) and 16 GB of RAM.
The Batches of Size 222 were simulated on the QUASM Simulator.

|Circuit|Shots|Dimension|Seconds per Batch|
|:------|----:|--------:|----------------:|
|Base|1024|4|4500|
|Efficient|1024|4|20|
|Efficient|512|4|12|
|Efficient|1024|4|80|

### Accuracy
Unfortunatley in terms of accuracy no meaningfull results could be archived. 
For the parameters given in the paper, the highest test accuracy score was **56%**.
The highest overall score archived when using a different set of parameters was **61%**.
The parameters are given below:

|Parameters|Value|
|:---------|----:|
|Epochs|200|
|Batchsize|222|
|a|0.05|
|b|0.01|
|A|5|
|s|0.602|
|t|0.101|
|𝛾|1|
|𝜂|1|
|𝜆|0|

The trainin courve looked as follows:
![Results](https://github.com/Gistbatch/quantum/blob/master/quantum/data/results.png)

### Problems
There are some problems regarding the implementation not given in the paper:
* Weight initialization
* Mapping of measurement results to labels

An additional problem is in the given explanation of the SPSA algorithm.
The implementation confuses the series a<sub>n</sub> and b<sub>n</sub>. In the original paper cited on SPSA (*Implementation of the simultaneous perturbation algorithm for stochastic optimization*) the roles of these series are swaped.
Additionally it seems the parameters given in the paper don't fulfill the required convergence criteria for SPSA.